<a href="https://colab.research.google.com/github/SinaQP/Athina/blob/main/Athina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Athina

In [ ]:
%pip install openai faiss-cpu langchain langchain-community langchain-openai --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [ ]:
from openai import OpenAI
import faiss
import numpy as np

In [ ]:
AVALAI_API_KEY = ""
AVALAI_BASE_URL = "https://api.avalai.ir/v1"
client = OpenAI(api_key=AVALAI_API_KEY, base_url=AVALAI_BASE_URL)

In [ ]:
def get_embedding(text):
    response = client.embeddings.create(input=text, model="text-embedding-ada-002")
    return np.array(response.data[0].embedding, dtype=np.float32)

In [ ]:
def store_data(question, answer):
    embedding = get_embedding(question)
    index.add(np.array([embedding]))  # Add embedding to FAISS index
    data_store[len(data_store)] = {"question": question, "answer": answer, "embedding": embedding}

In [ ]:
def search_similar_questions(query, top_k=5):
    query_embedding = get_embedding(query)
    distances, indices = index.search(np.array([query_embedding]), top_k)
    results = []
    for idx in indices[0]:
        if idx in data_store:
            results.append((data_store[idx]["question"], data_store[idx]["answer"]))
    return results


In [ ]:
dimension = 1536  # Dimension of OpenAI embeddings
index = faiss.IndexFlatL2(dimension)
data_store = {}

In [ ]:
# Store some example data
store_data("مراحل ایجاد حساب کاربری چیست؟", "ابتدا ایمیل خود را وارد کنید، سپس رمز عبور انتخاب کنید.")
store_data("چگونه یک حساب کاربری ایجاد کنم؟", "به صفحه ثبت‌نام بروید و اطلاعات خود را وارد کنید.")
store_data("سلام چطوری", "سلام من سینا قاسم پور هستم")
store_data("سلام", "سلام من سینا قاسم پور هستم")
store_data("صفحه عملکرد برای چیست", "سما در این عصر جدید تحول صفحه عملکرد را درست کردیم که برای شما و مدیریت سازمان شما کمک بفرمایید")
store_data("چگونه به صفحه عملکرد برم ؟", "شما برای وارد شدن به صفحه عملکرد اول باید روی دکمه عملکرد کارکنان کلید کنید و اطلاعات کاربری خود را وارد کنید")


In [ ]:
link_tag = "<LINK to='/working'>صفحه کاری</LINK> "
store_data("چگونه به صفحه کاری برم ؟", f"شما برای وارد شدن به صفحه کاری اول باید روی دکمه {link_tag} کلید کنید و اطلاعات کاربری خود را وارد کنید")


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
context_prompt = """
شما یک دستیار هوشمند هستید که به کاربران در سوالات مختلف کمک می‌کنید.
 با لحن دوستانه و کاربردی جواب دهید.
"""

def generate_response(user_query):
    # Initialize the chat model
    chat = ChatOpenAI(model="gpt-4o-mini", openai_api_key=AVALAI_API_KEY, base_url=AVALAI_BASE_URL, temperature=0.1)

    # Search for similar questions
    similar_questions = search_similar_questions(user_query)

    # Build the prompt text
    prompt_text = context_prompt + "\n\n"
    for q, a in similar_questions:
        prompt_text += f"سوال: {q}\nپاسخ: {a}\n\n"
    prompt_text += f"سوال جدید: {user_query}\nپاسخ:"

    # Create the message format for the model
    messages = [
        SystemMessage(content=context_prompt),
        HumanMessage(content=prompt_text)
    ]

    # Generate the response
    response = chat(messages)

    # Return the generated response
    return response.content


In [ ]:
result = generate_response("چگونه به صفحه کاری برم")
print(result)


برای رفتن به صفحه کاری، کافی است روی دکمه <LINK to='/working'>صفحه کاری</LINK> کلیک کنید و سپس اطلاعات کاربری خود را وارد کنید. اگر سوال دیگری دارید، خوشحال می‌شوم کمک کنم!


In [ ]:
print(result)

برای ایجاد حساب کاربری، به صفحه ثبت‌نام وب‌سایت یا اپلیکیشن مورد نظر بروید. سپس اطلاعات مورد نیاز مانند ایمیل، نام کاربری و رمز عبور را وارد کنید و بر روی دکمه ثبت‌نام کلیک کنید. در برخی موارد ممکن است نیاز به تأیید ایمیل یا شماره تلفن داشته باشید.
